In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

sns.set(style='white', context='notebook', palette='deep')

### Load data

In [20]:
df = pd.read_csv("./input/2018.csv")

In [23]:
df.head()

,מספר רכב,מספר הזמנה,קטגוריית שריון,ממוצע מרחק לנסיעה,דקות לחיוב,כתובת התחלה,כתובת סיום,תאריך ושעה התחלת שריון,תאריך ושעה סיום הזמנה
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל","הרב רובינשטיין 61, תל אביב יפו, ישראל",23/10/2017 0:08,23/10/2017 0:31
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל","ז'בוטינסקי 10, תל אביב יפו, ישראל",23/10/2017 0:09,23/10/2017 0:21
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל","רופין 32-34, תל אביב יפו, ישראל",23/10/2017 0:10,23/10/2017 0:15
3,63-906-81,60389,שריון חינמי,4,9,"שדרות מסריק 19, תל אביב יפו, ישראל","אונקלוס 3, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:23
4,60-589-55,60390,שריון חינמי,2,6,"הירקון 188, תל אביב יפו, ישראל","גורדון 83-85, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:21


In [29]:
df = df.rename({'מספר רכב': 'Car id', 'מספר הזמנה': 'Order number','קטגוריית שריון':'Category',
                 'ממוצע מרחק לנסיעה':'Avg distance','דקות לחיוב':'Billing minutes','כתובת התחלה':'Start Address',
                 'כתובת סיום':'End Address','תאריך ושעה התחלת שריון':'Start time','תאריך ושעה סיום הזמנה':'End time'},
                axis=1) 

In [30]:
df.head()

,Car id,Order number,Category,Avg distance,Billing minutes,Start Address,End Address,Start time,End time
0,46-525-81,60386,שריון חינמי,8,13,"גורדון 86-88, תל אביב יפו, ישראל","הרב רובינשטיין 61, תל אביב יפו, ישראל",23/10/2017 0:08,23/10/2017 0:31
1,48-437-81,60387,שריון חינמי,3,8,"יעקב אפטר 3, תל אביב יפו, ישראל","ז'בוטינסקי 10, תל אביב יפו, ישראל",23/10/2017 0:09,23/10/2017 0:21
2,63-887-81,60388,שריון חינמי,0,2,"רופין 32-34, תל אביב יפו, ישראל","רופין 32-34, תל אביב יפו, ישראל",23/10/2017 0:10,23/10/2017 0:15
3,63-906-81,60389,שריון חינמי,4,9,"שדרות מסריק 19, תל אביב יפו, ישראל","אונקלוס 3, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:23
4,60-589-55,60390,שריון חינמי,2,6,"הירקון 188, תל אביב יפו, ישראל","גורדון 83-85, תל אביב יפו, ישראל",23/10/2017 0:12,23/10/2017 0:21


### check for null and missing values

In [18]:
# Fill empty and NaNs values with NaN
df = df.fillna(np.nan)

In [19]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150002 entries, 0 to 150001
Data columns (total 9 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   מספר רכב                150002 non-null  object
 1   מספר הזמנה              150002 non-null  int64 
 2   קטגוריית שריון          150002 non-null  object
 3   ממוצע מרחק לנסיעה       150002 non-null  int64 
 4   דקות לחיוב              137847 non-null  object
 5   כתובת התחלה             150002 non-null  object
 6   כתובת סיום              136964 non-null  object
 7   תאריך ושעה התחלת שריון  150002 non-null  object
 8   תאריך ושעה סיום הזמנה   136356 non-null  object
dtypes: int64(2), object(7)
memory usage: 6.3+ MB


מספר רכב                      0
מספר הזמנה                    0
קטגוריית שריון                0
ממוצע מרחק לנסיעה             0
דקות לחיוב                12155
כתובת התחלה                   0
כתובת סיום                13038
תאריך ושעה התחלת שריון        0
תאריך ושעה סיום הזמנה     13646
dtype: int64